## 이미지 이동, 이름 변환

In [11]:
import os
import shutil
from pathlib import Path

# --- 1. 설정: 이 3개의 경로를 사용자의 환경에 맞게 수정하세요. ---

# ID가 적힌 텍스트 파일 경로 (예: "C:/Users/Penguin/Desktop/VQA/log.txt")
ID_LIST_FILE = "D:/VQA/cocoqa/test/img_ids.txt" 

# 원본 이미지 폴더 (000000...jpg 파일들이 있는 곳)
SOURCE_FOLDER = "D:/datasets/coco/images/train2017"

# 이미지를 옮길 대상 폴더 (새 이름으로 저장될 곳)
DESTINATION_FOLDER = "D:/VQA/cocoqa/test/images"

# --- ---------------------------------------------------- ---

def move_images_by_id(id_file, source_dir, dest_dir):
    print(f"ID 파일: {id_file}")
    print(f"원본 폴더: {source_dir}")
    print(f"대상 폴더: {dest_dir}")

    # 1. 대상 폴더가 없으면 생성
    os.makedirs(dest_dir, exist_ok=True)
    
    # 2. 텍스트 파일에서 ID 목록 읽기 (set으로 만들어 빠른 검색)
    try:
        with open(id_file, 'r') as f:
            # 공백이나 줄바꿈 문자를 제거하고 set에 저장
            id_set = {line.strip() for line in f if line.strip()}
        print(f"총 {len(id_set)}개의 ID를 텍스트 파일에서 읽었습니다.")
    except FileNotFoundError:
        print(f"오류: ID 파일 '{id_file}'을(를) 찾을 수 없습니다. 경로를 확인하세요.")
        return
    except Exception as e:
        print(f"ID 파일 읽기 오류: {e}")
        return

    # 3. 원본 폴더 순회 및 파일 이동
    moved_count = 0
    
    try:
        all_files = os.listdir(source_dir)
        print(f"원본 폴더에서 {len(all_files)}개의 파일을 스캔합니다...")
    except FileNotFoundError:
        print(f"오류: 원본 폴더 '{source_dir}'을(를) 찾을 수 없습니다. 경로를 확인하세요.")
        return
    
    for filename in all_files:
        # .jpg 파일만 대상으로 함
        if filename.lower().endswith('.jpg'):
            try:
                # 파일명에서 확장자(.jpg) 제거 (예: "000000000514")
                basename = filename.split('.')[0]
                
                # 'int'로 변환 후 'str'로 다시 변환하여 앞의 0 제거 (예: "514")
                id_without_zeros = str(int(basename))
                
                # 4. ID 목록(set)에 해당 ID가 있는지 확인
                if id_without_zeros in id_set:
                    
                    # 5. 새 파일 이름 정의 (예: "514.jpg")
                    new_filename = f"{id_without_zeros}.jpg"
                    
                    # 원본 경로와 대상 경로 설정
                    source_path = os.path.join(source_dir, filename)
                    destination_path = os.path.join(dest_dir, new_filename)
                    
                    # 6. 파일 이동 (shutil.move 사용)
                    shutil.move(source_path, destination_path)
                    
                    # (로그가 너무 많이 보이면 이 print문은 주석 처리)
                    # print(f"이동: {filename} -> {new_filename}") 
                    moved_count += 1
                    
            except ValueError:
                # 파일 이름이 숫자가 아닌 경우 (예: "readme.jpg")
                print(f"스킵: '{filename}'은(는) 숫자 형식의 이름이 아닙니다.")
            except Exception as e:
                print(f"파일 처리 중 오류 발생 ({filename}): {e}")

    print("\n--- 작업 완료 ---")
    print(f"총 {moved_count}개의 파일을 '{dest_dir}'(으)로 이동했습니다.")
    
    # ID 목록에는 있었지만 실제 파일이 없었던 경우
    if moved_count < len(id_set):
        print(f"참고: ID 목록의 {len(id_set) - moved_count}개 ID는 원본 폴더에 파일이 없었습니다.")

if __name__ == "__main__":
    move_images_by_id(ID_LIST_FILE, SOURCE_FOLDER, DESTINATION_FOLDER)

ID 파일: D:/VQA/cocoqa/test/img_ids.txt
원본 폴더: D:/datasets/coco/images/train2017
대상 폴더: D:/VQA/cocoqa/test/images
총 22879개의 ID를 텍스트 파일에서 읽었습니다.
원본 폴더에서 71994개의 파일을 스캔합니다...

--- 작업 완료 ---
총 20069개의 파일을 'D:/VQA/cocoqa/test/images'(으)로 이동했습니다.
참고: ID 목록의 2810개 ID는 원본 폴더에 파일이 없었습니다.


## qustions.json 파일 생성

In [4]:
import json
import os

# --- 1. 파일 경로 설정 ---

# (입력) 텍스트 파일 3개
QUESTIONS_TXT = "D:/VQA/cocoqa/test/questions.txt"
IMG_IDS_TXT = "D:/VQA/cocoqa/test/img_ids.txt"
ANSWERS_TXT = "D:/VQA/cocoqa/test/answers.txt"

# (출력) easyVQA 데이터셋 구조에 맞게 경로 설정
# (이 폴더가 없다면 스크립트가 자동으로 생성합니다)
TARGET_DIR = "D:/VQA/cocoqa/test/"
OUTPUT_JSON = os.path.join(TARGET_DIR, "questions.json")

# --- ------------------- ---

def create_questions_json():
    
    # 1. 출력 폴더 생성 (이미 있으면 무시)
    os.makedirs(TARGET_DIR, exist_ok=True)
    
    combined_data = []
    line_count = 0

    print(f"파일을 읽기 시작합니다:\n- {QUESTIONS_TXT}\n- {IMG_IDS_TXT}\n- {ANSWERS_TXT}")

    try:
        # 2. 세 개의 파일을 동시에 열기
        with open(QUESTIONS_TXT, 'r', encoding='utf-8') as f_q, \
             open(IMG_IDS_TXT, 'r', encoding='utf-8') as f_id, \
             open(ANSWERS_TXT, 'r', encoding='utf-8') as f_a:
            
            # 3. zip을 사용해 각 파일의 n번째 줄을 묶어서 순회
            for q_line, id_line, a_line in zip(f_q, f_id, f_a):
                
                # 4. 각 줄의 양쪽 공백/줄바꿈 문자 제거
                question = q_line.strip()
                image_id_str = id_line.strip()
                answer = a_line.strip()
                
                # (방어 코드) 빈 줄이 있으면 건너뛰기
                if not question or not image_id_str or not answer:
                    print(f"경고: {line_count + 1}번째 줄에 빈 데이터가 있어 건너뜁니다.")
                    continue
                
                try:
                    # 5. 이미지 ID를 정수(int)로 변환
                    image_id = int(image_id_str)
                    
                    # 6. 최종 리스트 형식 [질문, 답변, 이미지ID]로 추가
                    combined_data.append([question, answer, image_id])
                    line_count += 1
                    
                except ValueError:
                    print(f"경고: {line_count + 1}번째 줄의 이미지 ID '{image_id_str}'가 숫자가 아니어서 건너뜁니다.")

        print(f"\n총 {line_count}개의 데이터를 성공적으로 처리했습니다.")

        # 7. 리스트를 questions.json 파일로 저장
        print(f"데이터를 '{OUTPUT_JSON}' 파일로 저장합니다...")
        with open(OUTPUT_JSON, 'w', encoding='utf-8') as f_out:
            # json.dump를 사용해 리스트를 JSON 형식으로 저장
            json.dump(combined_data, f_out)
            
        print(f"--- 작업 완료! ---")
        print(f"'{OUTPUT_JSON}' 파일이 생성되었습니다.")

    except FileNotFoundError as e:
        print(f"오류: 파일을 찾을 수 없습니다. {e.filename}")
        print("스크립트가 텍스트 파일들과 동일한 위치에서 실행 중인지 확인하세요.")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    create_questions_json()

파일을 읽기 시작합니다:
- D:/VQA/cocoqa/test/questions.txt
- D:/VQA/cocoqa/test/img_ids.txt
- D:/VQA/cocoqa/test/answers.txt

총 38948개의 데이터를 성공적으로 처리했습니다.
데이터를 'D:/VQA/cocoqa/test/questions.json' 파일로 저장합니다...
--- 작업 완료! ---
'D:/VQA/cocoqa/test/questions.json' 파일이 생성되었습니다.


## Label 생성

In [3]:
import os

# --- 1. 경로 설정 ---

# (입력) 답변 원본 텍스트 파일
ANSWERS_TXT = "D:/VQA/cocoqa/train/answers.txt"

# (출력) 중복 제거된 답변 목록을 저장할 파일
OUTPUT_DIR = "D:/VQA/cocoqa/"
OUTPUT_TXT = os.path.join(OUTPUT_DIR, "labels.txt")

# --- ---------------- ---

def create_labels_from_answers():
    
    print(f"'{ANSWERS_TXT}' 파일을 읽어 중복 제거된 답변(레이블)을 추출합니다.")

    try:
        # 1. answers.txt 파일 읽기
        with open(ANSWERS_TXT, 'r', encoding='utf-8') as f:
            # 각 줄의 공백/줄바꿈 문자를 제거하고 set에 저장 (자동으로 중복 제거)
            unique_answers = {line.strip() for line in f if line.strip()}

        # 2. set을 리스트로 변환하고 알파벳 순으로 정렬
        sorted_labels = sorted(list(unique_answers))
        
        # 3. 출력 폴더 생성 (이미 있으면 무시)
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        # 4. 정렬된 레이블을 'labels.txt' 파일에 한 줄씩 저장
        print(f"총 {len(sorted_labels)}개의 고유한 레이블을 '{OUTPUT_TXT}' 파일에 저장합니다.")
        with open(OUTPUT_TXT, 'w', encoding='utf-8') as f:
            for label in sorted_labels:
                f.write(label + '\n')
        
        print("\n--- 작업 완료! ---")
        print(f"고유한 레이블 개수: {len(sorted_labels)}")
        print(f"목록은 '{OUTPUT_TXT}' 파일에서 확인할 수 있습니다.")

    except FileNotFoundError:
        print(f"오류: '{ANSWERS_TXT}' 파일을 찾을 수 없습니다.")
        print("스크립트와 같은 폴더에 파일이 있는지 확인하세요.")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    create_labels_from_answers()

'D:/VQA/cocoqa/train/answers.txt' 파일을 읽어 중복 제거된 답변(레이블)을 추출합니다.
총 430개의 고유한 레이블을 'D:/VQA/cocoqa/labels.txt' 파일에 저장합니다.

--- 작업 완료! ---
고유한 레이블 개수: 430
목록은 'D:/VQA/cocoqa/labels.txt' 파일에서 확인할 수 있습니다.
